<h2>Project 3: Na&iuml;ve Bayes and the Perceptron</h2>

<blockquote>
    <center>
    <img src="nb.png" width="200px" />
    </center>
      <p><cite><center>"All models are wrong, but some are useful."<br>
       -- George E.P. Box
      </center></cite></p>
</blockquote>

<h3>Introduction</h3>
<!--Aðalbrandr-->

<p>A&eth;albrandr is visiting America from Norway and has been having the hardest time distinguishing boys and girls because of the weird American names like Jack and Jane.  This has been causing lots of problems for A&eth;albrandr when he goes on dates. When he heard that Cornell has a Machine Learning class, he asked that we help him identify the gender of a person based on their name to the best of our ability.  In this project, you will implement Na&iuml;ve Bayes to predict if a name is male or female. </p>

<strong>How to submit:</strong> You can submit your code using the red <strong>Submit</strong> button above. This button will send any code below surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags below to the autograder, which will then run several tests over your code. By clicking on the <strong>Details</strong> dropdown next to the Submit button, you will be able to view your submission report once the autograder has completed running. This submission report contains a summary of the tests you have failed or passed, as well as a log of any errors generated by your code when we ran it.

Note that this may take a while depending on how long your code takes to run! Once your code is submitted you may navigate away from the page as you desire -- the most recent submission report will always be available from the Details menu.

<p><strong>Evaluation:</strong> Your code will be autograded for technical
correctness and--on some assignments--speed. Please <em>do not</em> change the names of any provided functions or classes within the code, or you will wreak havoc on the autograder. Furthermore, <em>any code not surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags will not be run by the autograder</em>. However, the correctness of your implementation -- not the autograder's output -- will be the final judge of your score.  If necessary, we will review and grade assignments individually to ensure that you receive due credit for your work.

<p><strong>Academic Integrity:</strong> We will be checking your code against other submissions in the class for logical redundancy. If you copy someone else's code and submit it with minor changes, we will know. These cheat detectors are quite hard to fool, so please don't try. We trust you all to submit your own work only; <em>please</em> don't let us down. If you do, we will pursue the strongest consequences available to us.

<p><strong>Getting Help:</strong> You are not alone!  If you find yourself stuck  on something, contact the course staff for help.  Office hours, section, and the <a href="https://piazza.com/class/iyag4nk2rsxsv">Piazza</a> are there for your support; please use them.  If you can't make our office hours, let us know and we will schedule more.  We want these projects to be rewarding and instructional, not frustrating and demoralizing.  But, we don't know when or how to help unless you ask.
  

<h3> Of boys and girls </h3>

<p> Take a look at the files <code>girls.train</code> and <code>boys.train</code>. For example with the unix command <pre>cat girls.train</pre> 
<pre>
...
Addisyn
Danika
Emilee
Aurora
Julianna
Sophia
Kaylyn
Litzy
Hadassah
</pre>
Believe it or not, these are all more or less common girl names. The problem with the current file is that the names are in plain text, which makes it hard for a machine learning algorithm to do anything useful with them. We therefore need to transform them into some vector format, where each name becomes a vector that represents a point in some high dimensional input space. </p>

<p>That is exactly what the following Python function <code>name2features</code> does: </p>

In [1]:
#<GRADED>
import numpy as np
#</GRADED>
import sys
# add p03 folder
sys.path.insert(0, './p03/')

%matplotlib inline

In [15]:
def hashfeatures(baby, B):
    v = np.zeros(B)
    for letter in baby:
        v[hash(letter) % B] = 1
    return v

In [16]:
def name2features(filename, B=128, LoadFile=True):
    """
    Output:
    X : n feature vectors of dimension B, (nxB)
    """
    # read in baby names
    if LoadFile:
        with open(filename, 'r') as f:
            babynames = [x.rstrip() for x in f.readlines() if len(x) > 0]
    else:
        babynames = filename.split('\n')
    n = len(babynames)
    X = np.zeros((n, B))
    for i in range(n):
        X[i,:] = hashfeatures(babynames[i], B)
    return X

It reads every name in the given file and converts it into a 128-dimensional feature vector. </p> 

<p>Can you figure out what the features are? (Understanding how these features are constructed will help you later on in the competition.)<br></p>

<p>We have provided you with a python function <code>genTrainFeatures</code>, which calls this script, transforms the names into features and loads them into memory. 

In [3]:
def genTrainFeatures(dimension=128):
    """
    function [x,y]=genTrainFeatures
    
    This function calls the python script "name2features.py" 
    to convert names into feature vectors and loads in the training data. 
    
    
    Output: 
    x: n feature vectors of dimensionality d [d,n]
    y: n labels (-1 = girl, +1 = boy)
    """
    
    # Load in the data
    Xgirls = name2features("girls.train", B=dimension)
    Xboys = name2features("boys.train", B=dimension)
    X = np.concatenate([Xgirls, Xboys])
    
    # Generate Labels
    Y = np.concatenate([-np.ones(len(Xgirls)), np.ones(len(Xboys))])
    
    # shuffle data into random order
    ii = np.random.permutation([i for i in range(len(Y))])
    
    return X[ii, :], Y[ii]

You can call the following command to load in the features and the labels of all boys and girls names. 

In [4]:
X,Y = genTrainFeatures(128)

<h3> The Na&iuml;ve Bayes Classifier </h3>

<p> The Na&iuml;ve Bayes classifier is a linear classifier based on Bayes Rule. The following questions will ask you to finish these functions in a pre-defined order. <br>
<strong>As a general rule, you should avoid tight loops at all cost.</strong></p>
<p>(a) Estimate the class probability P(Y) in 
<b><code>naivebayesPY</code></b>
. This should return the probability that a sample in the training set is positive or negative, independent of its features.
</p>



In [5]:
#<GRADED>
def naivebayesPY(x,y):
    """
    naivebayesPY(x,y) returns [pos,neg]

    Computation of P(Y)
    Input:
        x : n input vectors of d dimensions (nxd)
        y : n labels (-1 or +1) (nx1)

    Output:
    pos: probability p(y=1)
    neg: probability p(y=-1)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    y = np.concatenate([y, [-1,1]])
    n = len(y)
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    pos = ( np.sum(y) + n) / 2 / n
    neg = 1 - pos
    return (pos, neg)
#</GRADED>

pos,neg = naivebayesPY(X,Y)

<p>(b) Estimate the conditional probabilities P(X|Y) in 
<b><code>naivebayesPXY</code></b>
.  Use a <b>multinomial</b> distribution as model. This will return the probability vectors  for all features given a class label.
</p> 

In [6]:
#<GRADED>
def naivebayesPXY(x,y):
    """
    naivebayesPXY(x,y) returns [posprob,negprob]
    
    Input:
        x : n input vectors of d dimensions (nxd)
        y : n labels (-1 or +1) (nx1)
    
    Output:
    posprob: probability vector of p(x|y=1) (1xd)
    negprob: probability vector of p(x|y=-1) (1xd)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    n, d = x.shape
    x = np.concatenate([x, np.ones((2,d))])
    y = np.concatenate([y, [-1,1]])
    n, d = x.shape
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    Z=np.c_[x,y]
    Z=Z[np.lexsort(Z.T)]
    pos,neg=0,0
    for m in y:
        if m==-1:
            neg+=1
    Z=Z[:,:-1]
    negprob=Z[:neg].sum(axis=0)/(Z[:neg].sum())
    posprob=Z[neg:].sum(axis=0)/(Z[neg:].sum())
    return (posprob,negprob)
#</GRADED>
naivebayesPXY(X,Y)


(array([0.00228484, 0.0092917 , 0.00563595, 0.00152323, 0.00091394,
        0.00091394, 0.00700685, 0.0106626 , 0.00609292, 0.00654989,
        0.04843869, 0.02528561, 0.0166032 , 0.00594059, 0.00030465,
        0.00563595, 0.00258949, 0.0106626 , 0.02132521, 0.00913938,
        0.00487433, 0.00274181, 0.00319878, 0.00258949, 0.05529322,
        0.04569688, 0.03731912, 0.00060929, 0.00350343, 0.01279513,
        0.00167555, 0.01188119, 0.00182788, 0.00152323, 0.00822544,
        0.00426504, 0.00030465, 0.00258949, 0.00426504, 0.00304646,
        0.01309977, 0.00045697, 0.00441736, 0.00289414, 0.00152323,
        0.00167555, 0.00319878, 0.00380807, 0.00319878, 0.00182788,
        0.01309977, 0.0053313 , 0.00350343, 0.00137091, 0.00563595,
        0.00152323, 0.00502666, 0.00456969, 0.00076161, 0.00548363,
        0.00746382, 0.02223915, 0.00137091, 0.00350343, 0.00121858,
        0.00137091, 0.00030465, 0.00776847, 0.01020564, 0.04036558,
        0.00274181, 0.00350343, 0.00121858, 0.00

<p>(c) Solve for the log ratio, $\log\left(\frac{P(Y=1 | X = xtest)}{P(Y=-1|X= xtest)}\right)$, using Bayes Rule.
 Implement this in 
<b><code>naivebayes</code></b>.
</p>



In [7]:
#<GRADED>
def naivebayes(x,y,xtest):
    """
    naivebayes(x,y) returns logratio 
    
    Computation of log P(Y|X=x1) using Bayes Rule
    Input:
    x : n input vectors of d dimensions (nxd)
    y : n labels (-1 or +1)
    xtest: input vector of d dimensions (1xd)
    
    Output:
    logratio: log (P(Y = 1|X=xtest)/P(Y=-1|X=xtest))
    """
    
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    n, d = x.shape
    posprob, negprob=0,0
    for i in range(n):
        if y[i]==1 and (x[i]==xtest).all():
            posprob+=1
        if y[i]==-1 and (x[i]==xtest).all():
            negprob+=1      
    pos,neg =naivebayesPY(x,y)
    posprob= posprob 
    negprob= negprob 
    return np.log((posprob+1/n)/(negprob+1/n))
            
    

#</GRADED>
p=naivebayes(X,Y,X[0])

<p>(d) Naïve Bayes can also be written as a linear classifier. Implement this in 
<b><code>naivebayesCL</code></b>.

In [8]:
#<GRADED>
def naivebayesCL(x,y):
    """
    naivebayesCL(x,y) returns [w,b]
    Implementation of a Naive Bayes classifier
    Input:
    x : n input vectors of d dimensions (nxd)
    y : n labels (-1 or +1)

    Output:
    w : weight vector of d dimensions
    b : bias (scalar)
    """
    
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    n,d=x.shape
    x1,x2=naivebayesPXY(x,y)
    y1,y2=naivebayesPY(x,y)
    w= np.zeros(d)
    for i in range(d):
        w[i]=np.log(x1[i]/x2[i])
    b = np.log(y1/y2)
    return (w,b)
#</GRADED>

w,b=naivebayesCL(X,Y)
print(w,b)



[-4.37779035e-01 -2.13610368e-01 -7.63576740e-01  9.48515326e-01
  2.14546151e-01 -9.48604659e-01  7.66763568e-02  5.67147770e-01
  3.53808218e-01 -4.11267909e-01  6.41761940e-02 -4.35517443e-01
 -3.86014584e-02 -4.93868502e-01 -3.73240514e-01  5.98620069e-01
 -8.72231680e-01  8.79522455e-01 -3.19437808e-01  8.30732290e-01
  4.53438059e-01  8.43154810e-01  8.10147584e-02 -1.63437173e+00
 -2.15409977e-01  1.30205003e-01  5.49876183e-01  3.19906667e-01
 -1.99577020e-01 -4.56939533e-01  1.33150758e+00  5.82270931e-01
 -1.21926086e-01 -3.73240514e-01  1.38754555e-02  1.86375274e-01
  7.25371775e-01 -6.00297965e-01 -5.67396528e-01  2.55368146e-01
  2.96917148e-01  3.22245942e-02 -3.38149194e-01 -1.58830643e-01
 -1.50096963e-01 -1.34829490e-01  5.11797674e-01 -1.50096963e-01
 -5.87471840e-02  5.71221095e-01  5.94751592e-01 -1.73627460e-01
  3.22245942e-02 -6.60922586e-01  1.15823586e+00  3.22245942e-02
 -1.34829490e-01 -1.50096963e-01 -1.50096963e-01 -4.27307735e-01
 -4.32663934e-01  1.48296

<p>(e) Implement 
<b><code>classifyLinear</code></b>
 that applies a linear weight vector and bias to a set of input vectors and outputs their predictions.  (You can use your answer from the previous project.)
 
 

In [9]:
#<GRADED>
def classifyLinear(x,w,b):
    """
    classifyLinear(x,w,b) returns preds
    
    Make predictions with a linear classifier
    Input:
    x : n input vectors of d dimensions (nxd)
    w : weight vector of d dimensions
    b : bias (optional)
    
    Output:
    preds: predictions
    """
    
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    pred = np.sign(x.dot(w)+b)
    return pred
#</GRADED>

print('Training error: %.2f%%' % (100 *(classifyLinear(X, w, b) != Y).mean()))

Training error: 22.08%


You can now test your code with the following interactive name classification script:

In [ ]:
DIMS = 128
print('Loading data ...')
X,Y = genTrainFeatures(DIMS)
print('Training classifier ...')
w,b=naivebayesCL(X,Y)
error = np.mean(classifyLinear(X,w,b) != Y)
print('Training error: %.2f%%' % (100 * error))

while True:
    print('Please enter your name>')
    yourname = input()
    if len(yourname) < 1:
        break
    xtest = name2features(yourname,B=DIMS,LoadFile=False)
    pred = classifyLinear(xtest,w,b)[0]
    if pred > 0:
        print("%s, I am sure you are a nice boy.\n" % yourname)
    else:
        print("%s, I am sure you are a nice girl.\n" % yourname)

Loading data ...
Training classifier ...
Training error: 22.08%
Please enter your name>


<h3> Feature Extraction (Competition)</h3>

<p>(e) (<b>optional</b>) As always, this programming project also includes a competition.  We will rank all submissions by how well your Na&iuml;ve Bayes classifier performs on a secret test set. If you want to improve your classifier modify <code>name2features2</code> below.   The automatic reader will use your Python script to extract features and train your classifier on the same names training set by calling the function with only one argument--the name of a file containing a list of names.  The given implementation is the same as the given <code>name2features</code> above.
</p>
  

In [2]:
#<GRADED>
def hashfeatures(baby, B):
    v = np.zeros(B)
    baby.lower()
    v[hash(len(baby))] = 1
    for i in range(1,4):
        v[hash("suffix"+baby[max(-i,-len(baby)):]) % B] = 1 
        v[hash("middle"+baby[max(int(len(baby)/2)-1,0):min(int(len(baby)/2)+1,len(baby))]) % B] = 1
        v[hash("prelix"+baby[:min(i,len(baby))]) % B] = 1
        
    return v


def name2features2(filename, B=50000, LoadFile=True):
    """
    Output:
    X : n feature vectors of dimension B, (nxB)
    """
    # read in baby names
    if LoadFile:
        with open(filename, 'r') as f:
            babynames = [x.rstrip() for x in f.readlines() if len(x) > 0]
    else:
        babynames = filename.split('\n')
    n = len(babynames)
    X = np.zeros((n, B))
    for i in range(n):
        X[i,:] = hashfeatures(babynames[i], B)
    return X


#</GRADED>

<h4>Credits</h4>
  Parts of this webpage were copied from or heavily inspired by John DeNero's and Dan Klein's (awesome) <a href="http://ai.berkeley.edu/project_overview.html">Pacman class</a>. The name classification idea originates from <a href="http://nickm.com">Nick Montfort</a>.